# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weathercsv_df = pd.read_csv('../WeatherPy/output_data/weather_data.csv', index_col=0)

weathercsv_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Ixtapa,MX,20.7000,-105.2000,59.00,94,40,3.44,1612795091
1,Bokovskaya,RU,49.2290,41.8306,18.97,82,100,7.61,1612795091
2,Severo-Kuril'sk,RU,50.6789,156.1250,18.68,93,87,6.20,1612795091
3,Codrington,AU,-38.2667,141.9667,54.36,87,23,9.66,1612795091
4,Thompson,CA,55.7435,-97.8558,-25.60,68,90,14.97,1612795091
...,...,...,...,...,...,...,...,...,...
530,Ibrā’,OM,22.6906,58.5334,70.30,41,100,8.23,1612795091
531,Beisfjord,NO,68.3751,17.5988,30.20,86,40,3.44,1612795091
532,Severodvinsk,RU,64.5635,39.8302,-13.00,77,20,2.24,1612795091
533,Turukhansk,RU,65.8167,87.9833,4.82,94,100,7.94,1612795091


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = weathercsv_df[["Lat", "Lng"]].astype(float)
humidity = weathercsv_df['Humidity']
fig = gmaps.figure(zoom_level=2, center=(15, 0 ), layout={'width': '1100px', 'height': '600px'} )
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='600px', width='1100px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# weathercsv_df.dtypes

hotel_df = weathercsv_df.loc[weathercsv_df['Max Temp'] >= 75]
hotel_df = hotel_df.loc[hotel_df['Max Temp'] <= 100]
hotel_df = hotel_df.loc[hotel_df['Cloudiness'] <= 25]
hotel_df = hotel_df.loc[hotel_df['Wind Speed'] < 10]

hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
6,Bredasdorp,ZA,-34.5322,20.0403,82.40,44,0,9.22,1612795091
16,Tessalit,ML,20.1986,1.0114,83.53,13,0,6.24,1612795091
38,Hermanus,ZA,-34.4187,19.2345,78.01,67,0,7.00,1612795091
41,Bubaque,GW,11.2833,-15.8333,78.57,45,5,7.70,1612795091
58,Padang,ID,-0.9492,100.3543,78.80,89,20,2.30,1612795091
108,Cap Malheureux,MU,-19.9842,57.6142,82.99,73,0,5.99,1612795091
114,Kavaratti,IN,10.5669,72.6420,82.80,58,10,3.02,1612795091
126,Cidreira,BR,-30.1811,-50.2056,76.28,61,0,8.52,1612795091
166,Taoudenni,ML,22.6783,-3.9836,78.46,16,0,5.91,1612795091
193,Koungou,YT,-12.7336,45.2042,80.60,78,0,9.22,1612795091


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df['Hotel Name'] = ''

hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
6,Bredasdorp,ZA,-34.5322,20.0403,82.40,44,0,9.22,1612795091,
16,Tessalit,ML,20.1986,1.0114,83.53,13,0,6.24,1612795091,
38,Hermanus,ZA,-34.4187,19.2345,78.01,67,0,7.00,1612795091,
41,Bubaque,GW,11.2833,-15.8333,78.57,45,5,7.70,1612795091,
58,Padang,ID,-0.9492,100.3543,78.80,89,20,2.30,1612795091,
108,Cap Malheureux,MU,-19.9842,57.6142,82.99,73,0,5.99,1612795091,
114,Kavaratti,IN,10.5669,72.6420,82.80,58,10,3.02,1612795091,
126,Cidreira,BR,-30.1811,-50.2056,76.28,61,0,8.52,1612795091,
166,Taoudenni,ML,22.6783,-3.9836,78.46,16,0,5.91,1612795091,
193,Koungou,YT,-12.7336,45.2042,80.60,78,0,9.22,1612795091,


In [6]:
search_type = 'hotel'
radius = 5000
params = {
    "radius": radius,
    "types": search_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    hotels = requests.get(base_url, params=params)
    
    hotels = hotels.json()
    
    print("Retrieving next hotel")
    
    try:
        print(f'There is a hotel called {hotels["results"][1]["name"]}')
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][1]['name']
        
    except (KeyError, IndexError):
        print('No Hotel Available.')
        hotel_df.loc[index, 'Hotel Name'] = np.nan
        pass
    
    time.sleep(1)

Retrieving next hotel
There is a hotel called Bredasdorp Country Manor
Retrieving next hotel
There is a hotel called Tessalit Referral Health Centre
Retrieving next hotel
There is a hotel called Misty Waves Boutique Hotel
Retrieving next hotel
There is a hotel called Casa Dora, Bubaque
Retrieving next hotel
There is a hotel called Plaza Andalas
Retrieving next hotel
There is a hotel called Pereybere Hotel & Spa
Retrieving next hotel
There is a hotel called All India Radio Kavaratti
Retrieving next hotel
There is a hotel called Diego Imóveis
Retrieving next hotel
There is a hotel called Ruines du fort de Taoudénit
Retrieving next hotel
There is a hotel called Villa Netibor
Retrieving next hotel
There is a hotel called Asamblea de Cooperacion por la Paz (Senegal)
Retrieving next hotel
There is a hotel called Amerik HOTEL
Retrieving next hotel
There is a hotel called Banco Provincial
Retrieving next hotel
There is a hotel called Salinas Park Resort Maçarico AP101
Retrieving next hotel
The

In [7]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
6,Bredasdorp,ZA,-34.5322,20.0403,82.40,44,0,9.22,1612795091,Bredasdorp Country Manor
16,Tessalit,ML,20.1986,1.0114,83.53,13,0,6.24,1612795091,Tessalit Referral Health Centre
38,Hermanus,ZA,-34.4187,19.2345,78.01,67,0,7.00,1612795091,Misty Waves Boutique Hotel
41,Bubaque,GW,11.2833,-15.8333,78.57,45,5,7.70,1612795091,"Casa Dora, Bubaque"
58,Padang,ID,-0.9492,100.3543,78.80,89,20,2.30,1612795091,Plaza Andalas
108,Cap Malheureux,MU,-19.9842,57.6142,82.99,73,0,5.99,1612795091,Pereybere Hotel & Spa
114,Kavaratti,IN,10.5669,72.6420,82.80,58,10,3.02,1612795091,All India Radio Kavaratti
126,Cidreira,BR,-30.1811,-50.2056,76.28,61,0,8.52,1612795091,Diego Imóveis
166,Taoudenni,ML,22.6783,-3.9836,78.46,16,0,5.91,1612795091,Ruines du fort de Taoudénit
193,Koungou,YT,-12.7336,45.2042,80.60,78,0,9.22,1612795091,Villa Netibor


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=[f'Hotel Information: {hotel}' for hotel in hotel_info]
)

fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', width='1100px'))